In [6]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_ds = train_gen.flow_from_directory(
    './data/train/',
    target_size=(150, 150), # Size of the image
    batch_size=32  # How many image at once we need
)

test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_ds = test_gen.flow_from_directory(
    './data/train/',
    target_size=(150, 150), # Size of the image
    batch_size=32  # How many image at once we need
)

In [7]:
def make_model(
                filters = 32,
                dense_inner_size = 64,
                lr = 0.002,
                momentum = 0.8,
                metrics = ['accuracy'],
                loss = 'mean squared error'
                ):
    inputs = keras.Input(shape=(200,200,3))
    cvl_layers = Conv2D(filters=filters, kernel_size=(3,3),activation='relu')(inputs)
    feature_map = keras.layers.MaxPooling2D(pool_size=(2,2))(cvl_layers)
    vectors = Flatten()(feature_map)
    dense_layers = keras.layers.Dense(dense_inner_size, activation='relu')(vectors)
    output = keras.layers.Dense(1)(dense_layers)
    model = keras.Model(inputs,outputs)
    optimizer = SGD(lr=lr, momentum=momentum)
    model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=['accuracy']
    )
    return model


In [ ]:
scores = {}

for loss in ['mean squared error',
            'binary crossentropy',
            'categorical crossentropy',
            'cosine similarity']:
    model = make_model(loss=loss)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[loss] = history.history
    print()
    print()

